In [1]:
# BASIC IMPORTS OF ALL LIBRARIES
'''
Glove Imports are 300-dim from GENSIM
Pickle saves all the datasets
'''
import pygraphviz
from nltk import pos_tag
import csv
import re
import numpy as np
from nltk.stem import WordNetLemmatizer
import matplotlib.pyplot as plt
from collections import defaultdict
import networkx as nx
from copy import deepcopy
import pickle
from entity_groups import *
from graphviz import Source
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist 
import gensim.downloader as api

wnl = WordNetLemmatizer()
word_vectors = api.load("glove-wiki-gigaword-300")

In [97]:
# junk verbs that are generalizably illegal
junk_rel = ["be","have","do","let","","make","tell","say","describe","decide","want","name","know","think","try","become","oneday","put","come",'see', 'need', 'look', 'help', 'come', 'take', 'get', 'put', 'pick', 'turn', 'go', 'stand', 'give', 'notice', 'use',"get","start"]

# hypothetical words that are generally useless / qualifiers
junk_words = ["would","could","should","maybe","perhaps","think","might","assume","claim","this"]

# negative sentences
junk_words.extend(['n\'t',"not"])
print(junk_words)

# perspective actants that are pointless
junk = ["you","i","we","the","it","he","she","steinbeck","people","author",
        "book","me","steinback","him","her","their","this","","shelley","mary","harper","lee","tolkien"]

['would', 'could', 'should', 'maybe', 'perhaps', 'think', 'might', 'assume', 'claim', 'this', "n't", 'not']


In [98]:
DATE = "0413"
file_name = "OS_News/"
text = "OS_News"

In [99]:
import pandas as pd
results_path = "Results" + DATE + "/" + text
adj_mat_path = "Precedence_Matrix/" + text
abs_adj_mat_path = "AbsPrecedence_Matrix/" + text
dic_path = "Dictionary/" + text + ".pickle"
ner_file = "../Covid_19_Data/Extracted_Data/OS_News/" + "into_relex_final_relations_-1.csv"
entity_file = "../Covid_19_Data/Extracted_Data/OS_News/" + "df_ent_final_ranking.csv"
S_use = {}
# fill this in

places = []
ent_file = pd.read_csv(entity_file)
for row in ent_file.head(n=20).iterrows():
    if row[1]['type'] == "GPE":
        places.append(row[1]['entity'])
    S_use[row[1]['entity']] = row[1]['entity']

fileName = ner_file

In [100]:
actants_5g = ["5g","radio","transmitters","towers","radiation","army","waves","conspiracy"]
for a in actants_5g:
    S_use[a] = a

In [101]:
# HELPER FUNCTION
def hypothetical_sentence(sent):
    '''
    Removes sentences that have the hypothetical list of words in them. 
    Does not offer information about sequencing.
    '''
    for word in sent.split():
        for ground_word in junk_words:
            if ground_word in word:
                return True
    return False

In [102]:
trajectories = []
same_post = 0
traj = ["START"]

with open(fileName) as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')

    for row in readCSV:
        FLAG = 0
        sentence_num = row[1]
        sub_r = ""
        obj_r = ""
        sub_verb = ""

        if hypothetical_sentence(row[2]):
            continue

        try:
            sub = re.search(r"\{(\w+)\}", row[3]).group(1).lower()
            rel = wnl.lemmatize(re.search(r"\{(\w+)\}", row[4]).group(1).lower(), pos='v')
            obj = re.search(r"\{(\w+)\}", row[5]).group(1).lower()
        except:
            continue

        words = row[2].split()
        rel_words = []

        if rel == None or obj == None or sub == None:
            continue
        
        if same_post == row[0]:
            pass
        else:
            same_post = row[0]
            traj.append("TERMINATE")
            trajectories.append(traj)
            traj = ["START"]

        try:
            sub_r = S_use[sub]
        except:
            sub_r = sub
            FLAG += 1
        
        try:
            obj_r = S_use[obj]
        except:
            obj_r = obj
            FLAG += 1

        sub_verb = ""

        try:
            sub_verb = C_use[rel]
        except:
            sub_verb = rel

        if sub_r == obj_r or sub_r in junk or obj_r in junk or sub_verb in junk_rel \
                                or FLAG > 0 or sub_r.isdigit() or obj_r.isdigit():
            continue

        traj.append(sub_verb)
        traj.append(sub_r + "_" + obj_r + "_" + sentence_num)

# if we missed out on last post
if traj != "START":
    traj.append("TERMINATE")
    trajectories.append(traj)

In [103]:
print(trajectories)

ct', 'government_virus_34', 'contain', 'government_virus_34', 'TERMINATE'], ['START', 'TERMINATE'], ['START', 'TERMINATE'], ['START', 'cause', 'coronavirus_covid19_2', 'begin', 'outbreak_china_2', 'cause', 'coronavirus_covid19_2', 'TERMINATE'], ['START', 'invest', 'china_wuhan_32', 'TERMINATE'], ['START', 'cause', 'coronavirus_covid19_18', 'cause', 'coronavirus_covid19_18', 'TERMINATE'], ['START', 'batter', 'trump_outbreak_15', 'account', 'italy_cases_28', 'confirm', 'us_cases_32', 'TERMINATE'], ['START', 'TERMINATE'], ['START', 'TERMINATE'], ['START', 'TERMINATE'], ['START', 'TERMINATE'], ['START', 'TERMINATE'], ['START', 'TERMINATE'], ['START', 'TERMINATE'], ['START', 'TERMINATE'], ['START', 'TERMINATE'], ['START', 'TERMINATE'], ['START', 'TERMINATE'], ['START', 'enter', 'virus_china_7', 'TERMINATE'], ['START', 'TERMINATE'], ['START', 'TERMINATE'], ['START', 'TERMINATE'], ['START', 'TERMINATE'], ['START', 'TERMINATE'], ['START', 'TERMINATE'], ['START', 'TERMINATE'], ['START', 'TERMIN

In [104]:
def find_mat(trajectories):
    rev_dict = {}
    precedence_matrix = np.zeros((50000,50000))
    dictionary_of_labels = {"START":0, "TERMINATE":1}
    counter_of_labels = defaultdict(int)
    max_rows = 1
    max_cols = 1
    counter = 2

    for trajectory in trajectories:

        triplets = ["START"]
        index = 1

        while index < len(trajectory)-2:
            triplets.append(trajectory[index] + '_' + trajectory[index + 1])
            index += 2

        triplets.append("TERMINATE")

        for key,_ in enumerate(triplets[1:-1]):
            removed_triplets = triplets[key+1].split("_")
            if len(removed_triplets) > 1:
                subject_ = removed_triplets[1]
                object_ = removed_triplets[2]
                if subject_ not in dictionary_of_labels:
                    dictionary_of_labels[subject_] = counter
                    counter += 1
                if object_ not in dictionary_of_labels:
                    dictionary_of_labels[object_] = counter
                    counter += 1
        
        triplet_temp = []

        # find uniques
        for trip in triplets:
            try:
                removed_triplet = trip.rsplit("_",1)[0]
            except:
                removed_triplet = trip

            if removed_triplet not in [t.rsplit("_",1)[0] for t in triplet_temp]:
                triplet_temp.append(trip)

        triplets = triplet_temp

        # for trip in triplets:
        #     counter_of_labels[trip] += 1

        alpha = 1.0

        for key in range(1,len(triplets)):
            removed_triplets = triplets[key].split("_")
            try:
                subject_ = removed_triplets[1]
                object_ = removed_triplets[2]
                precedence_matrix[dictionary_of_labels[subject_]]\
                                            [dictionary_of_labels[object_]] += 1.0
            except:
                pass


    precedence_matrix = precedence_matrix[:counter,:counter]
    for key in dictionary_of_labels:
        rev_dict[dictionary_of_labels[key]] = key

    return precedence_matrix, dictionary_of_labels, rev_dict

In [105]:
actants_5g = ["5g","radio","transmitters","towers","radiation","army","waves","conspiracy"]
i = 0
while i < len(trajectories):
    sub_t = trajectories[0:i+10]
    mat, dic, rev_dict = find_mat(sub_t)

    G = nx.from_numpy_matrix(mat, create_using=nx.DiGraph)
    DISP_THRESH = 2

    bb = {}
    for index, val in np.ndenumerate(mat):
        if val > 1:
            bb[index] = {"color":"red", "penwidth":2}
        else:
            bb[index] = {"color":"blue"}
    nx.set_edge_attributes(G, bb)

    bb1 = {}
    for j in range(mat.shape[0]):
        if rev_dict[j] in actants_5g:
            bb1[j] = {"color":"red", "penwidth":4}
        else:
            bb1[j] = {"color":"blue"}

    nx.set_node_attributes(G, bb1)

    bb2 = {}
    for j in range(mat.shape[0]):
        if rev_dict[j] in places:
            bb2[j] = {"color":"green", "penwidth":4}
        else:
            bb2[j] = {"color":"blue"}
    nx.set_node_attributes(G, bb2)

    H = nx.relabel_nodes(G, rev_dict)
    #H.remove_nodes_from(list(nx.isolates(H)))
    A = nx.nx_agraph.to_agraph(H)

    # for i in range(number_of_steps):
    #     same_priority = []
    #     for j in range(len(time_step)):
    #         if time_step[j] == i:
    #             same_priority.append(diction[j])
    #     one = A.add_subgraph(same_priority,rank='same')

    A.layout('dot')
    A.draw("Results0413/Actant_TimeSeries/step" + str(i) + '.png')

    print(i)
    i = i + 10

        

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
1190
1200
1210
1220
1230
1240
1250
1260
1270
1280
1290
1300
1310
1320
1330
1340
1350
1360
1370
1380
1390
1400
1410
1420
1430
1440
1450
1460
1470
1480
1490
1500
1510
1520
1530
1540
1550
1560
1570
1580
1590
1600
1610
1620
1630
1640
1650
1660
1670
1680
1690
1700
1710


In [106]:
print(rev_dict)

{0: 'START', 1: 'TERMINATE', 2: 'china', 3: 'outbreak', 4: 'cases', 5: 'wuhan', 6: 'virus', 7: 'government', 8: 'uk', 9: 'coronavirus', 10: 'beijing', 11: 'chinese', 12: 'japan', 13: 'us', 14: 'trump', 15: 'italy', 16: 'covid19', 17: 'iran', 18: 'conspiracy', 19: 'army', 20: 'waves', 21: 'radiation'}
